In [1]:
## Potting for HRRR data from AWS (only after 2014)

In [3]:
from herbie import Herbie, FastHerbie, wgrib2
import pandas as pd
import xarray as xr
import geojson

import s3fs
import metpy
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

ModuleNotFoundError: No module named 's3fs'

In [ ]:
date = '20211016'
hour = '21'
var = 'TMP'
level = 'surface'
url1 = 's3://hrrrzarr/sfc/' + date + '/' + date + '_' + hour + 'z_anl.zarr/' + level + '/' + var + '/' + level
url2 = 's3://hrrrzarr/sfc/' + date + '/' + date + '_' + hour + 'z_anl.zarr/' + level + '/' + var
print(url1)
print(url2)

In [ ]:
fs = s3fs.S3FileSystem(anon=True)
file1 = s3fs.S3Map(url1, s3=fs)
file2 = s3fs.S3Map(url2, s3=fs)

In [ ]:
ds = xr.open_mfdataset([file1,file2], engine='zarr')

In [ ]:
ds

In [ ]:
lon1 = 262.5
lat1 = 38.5
slat = 38.5
projData= ccrs.LambertConformal(central_longitude=lon1,
                             central_latitude=lat1,
                             standard_parallels=[slat,slat],globe=ccrs.Globe(semimajor_axis=6371229,
                                        semiminor_axis=6371229))




In [ ]:
ds.coords

In [ ]:
airTemp = ds.TMP

In [ ]:
airTemp

In [ ]:
# Dask
airTemp = airTemp.metpy.convert_units('degC')
airTemp

In [ ]:
x = airTemp.projection_x_coordinate
y = airTemp.projection_y_coordinate

In [ ]:
# Visualize data
airTemp.plot(figsize=(11,8.5))

In [ ]:
airTemp.min()

In [ ]:
minTemp = airTemp.min().compute()
maxTemp = airTemp.max().compute()
minTemp.values, maxTemp.values

In [ ]:
fint = np.arange(np.floor(minTemp.values),np.ceil(maxTemp.values) + 2, 2)
fint

In [ ]:
latN = 50.4
latS = 24.25
lonW = -123.8
lonE = -71.2

res = '50m'

fig = plt.figure(figsize=(18,12))
ax = plt.subplot(1,1,1,projection=projData)
ax.set_extent ([lonW,lonE,latS,latN],crs=ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE.with_scale(res))
ax.add_feature(cfeature.STATES.with_scale(res))

# Add the title
tl1 = str('HRRR temperature surface ($^\circ$C)')
tl2 = str('Analysis valid at: '+ hour + '00 UTC ' + date  )
plt.title(tl1+'\n'+tl2,fontsize=16)

# Contour fill
CF = ax.contourf(x,y,airTemp,levels=fint,cmap=plt.get_cmap('coolwarm'))
# Make a colorbar for the ContourSet returned by the contourf call.
cbar = fig.colorbar(CF,shrink=0.5)
cbar.set_label(r'Temperature Surface($^\circ$C)', size='large')